# Flipkart EDA assignment

## TODO: Upload csv file before moving to next

In [0]:
from pyspark.sql import SparkSession

spark = SparkSession.builder.appName("Flipkart Data Engineering").getOrCreate()
file_path = '/FileStore/tables/Flipkart.csv'
flipkart_df = spark.read.csv(file_path, header=True, inferSchema=True)

In [0]:
flipkart_df.printSchema()
flipkart_df.count()

root
 |-- id: integer (nullable = true)
 |-- title: string (nullable = true)
 |-- Rating: double (nullable = true)
 |-- maincateg: string (nullable = true)
 |-- platform: string (nullable = true)
 |-- actprice1: integer (nullable = true)
 |-- norating1: integer (nullable = true)
 |-- noreviews1: integer (nullable = true)
 |-- star_5f: integer (nullable = true)
 |-- star_4f: integer (nullable = true)
 |-- star_3f: integer (nullable = true)
 |-- star_2f: integer (nullable = true)
 |-- star_1f: integer (nullable = true)
 |-- fulfilled1: integer (nullable = true)

Out[2]: 5244

In [0]:
flipkart_df.show(5)

+-----+--------------------+------+---------+--------+---------+---------+----------+-------+-------+-------+-------+-------+----------+
|   id|               title|Rating|maincateg|platform|actprice1|norating1|noreviews1|star_5f|star_4f|star_3f|star_2f|star_1f|fulfilled1|
+-----+--------------------+------+---------+--------+---------+---------+----------+-------+-------+-------+-------+-------+----------+
| 2242|Casuals For Men  ...|   3.8|      Men|Flipkart|      999|    27928|      3543|  14238|   4295|   3457|   1962|   3976|         1|
|20532|Women Black Flats...|   3.9|    Women|Flipkart|      499|     3015|       404|   1458|    657|    397|    182|    321|         1|
|10648|Women Gold Wedges...|   3.9|    Women|Flipkart|      999|      449|        52|    229|     70|     71|     33|     46|         1|
|20677|Men's Height Incr...|   3.9|      Men|Flipkart|     2999|      290|        40|    141|     51|     49|     17|     32|         1|
|12593|Loafers For Men  ...|   3.9|      

In [0]:
# Display all category names
flipkart_df.select("maincateg").distinct().show()

+---------+
|maincateg|
+---------+
|     null|
|      Men|
|    Women|
+---------+



In [0]:
# Filter products with rating > 4.5 and more than 1000 reviews

flipkart_df.select("title","Rating","noreviews1").filter((flipkart_df["Rating"] > 4) & (flipkart_df["noreviews1"] > 100)).show() 

+--------------------+------+----------+
|               title|Rating|noreviews1|
+--------------------+------+----------+
|Women Black Wedge...|   4.2|       487|
|Denill Ankle Leng...|   4.1|       961|
|     SFG-23 Slippers|   4.1|      1598|
|White casual, gym...|   4.2|       381|
|Synthetic Leather...|   4.2|       182|
|Running Shoes For...|   4.1|      8254|
|SM-484 Casuals Fo...|   4.2|       436|
|Women Black Heels...|   4.1|       182|
|Women Brown Wedge...|   4.1|       178|
|GO WALK 5-SPARROW...|   4.3|       106|
|crocs Unisex's Ra...|   4.3|       661|
|Men Grey Clogs Sa...|   4.1|       328|
|Men Navy Sports S...|   4.2|      2103|
|Women Black Wedge...|   4.2|       487|
|Sneakers For Men ...|   4.1|       597|
|Men Black Clogs S...|   4.2|      1098|
|Walking Shoes For...|   4.1|       938|
|Mojaris For Men  ...|   4.1|       106|
|Women Pink Wedges...|   4.2|       232|
|              Slides|   4.1|       141|
+--------------------+------+----------+
only showing top

In [0]:
# Display Products in 'Men' category that are fulfilled

flipkart_df.filter((flipkart_df["maincateg"] == "Men") & (flipkart_df["fulfilled1"] == 1)).show()



+-----+--------------------+------+---------+--------+---------+---------+----------+-------+-------+-------+-------+-------+----------+
|   id|               title|Rating|maincateg|platform|actprice1|norating1|noreviews1|star_5f|star_4f|star_3f|star_2f|star_1f|fulfilled1|
+-----+--------------------+------+---------+--------+---------+---------+----------+-------+-------+-------+-------+-------+----------+
| 2242|Casuals For Men  ...|   3.8|      Men|Flipkart|      999|    27928|      3543|  14238|   4295|   3457|   1962|   3976|         1|
|20677|Men's Height Incr...|   3.9|      Men|Flipkart|     2999|      290|        40|    141|     51|     49|     17|     32|         1|
|10680|Combo Pack Of 2 C...|   3.9|      Men|Flipkart|     2400|      824|       105|    386|    199|    106|     61|     72|         1|
|11579|MOCCASSIN Slip On...|   3.8|      Men|Flipkart|     2299|     3066|       392|   1304|    803|    462|    189|    308|         1|
|12260|Long OfficeWear C...|   3.9|      

In [0]:
# Dsiplay number of products per category
from pyspark.sql.functions import count

flipkart_df.groupBy("maincateg").agg(count("title")).show()

+---------+------------+
|maincateg|count(title)|
+---------+------------+
|     null|          67|
|      Men|        2145|
|    Women|        3032|
+---------+------------+



In [0]:
# Display Average rating per category
from pyspark.sql.functions import avg
flipkart_df.groupBy("maincateg").agg(avg("Rating")).show()





+---------+------------------+
|maincateg|       avg(Rating)|
+---------+------------------+
|     null|             4.225|
|      Men|3.9662970106075246|
|    Women|4.0416582064297915|
+---------+------------------+



In [0]:
# Dsiplay Category with highest average number of reviews
flipkart_df.groupBy("maincateg").agg(avg("noreviews1")).orderBy("avg(noreviews1)",ascending=False).show(1)

+---------+-----------------+
|maincateg|  avg(noreviews1)|
+---------+-----------------+
|      Men|759.6554778554779|
+---------+-----------------+
only showing top 1 row



In [0]:
# Top 5 products with highest price. display product name and price
flipkart_df.select("title","actprice1").orderBy("actprice1",ascending=False).show(5)



+--------------------+---------+
|               title|actprice1|
+--------------------+---------+
|GEL-QUANTUM 360 K...|    15999|
|Women Orange Spor...|    12999|
|Hybrid Astro Runn...|    10999|
|GT-2000 5 (D) Run...|    10999|
|DEERUPT RUNNER W ...|     9999|
+--------------------+---------+
only showing top 5 rows



In [0]:
# Display Min, max, and avg price per category
from pyspark.sql.functions import avg,min,max
flipkart_df.groupBy("maincateg").agg(avg("actprice1").alias("Avg Price"),min("actprice1").alias("Min Price"),max("actprice1").alias("Max Price")).show()



+---------+------------------+---------+---------+
|maincateg|         Avg Price|Min Price|Max Price|
+---------+------------------+---------+---------+
|     null|1574.7313432835822|      499|     5999|
|      Men|  1455.86013986014|      139|    10999|
|    Women|1319.7081134564644|      162|    15999|
+---------+------------------+---------+---------+



In [0]:
from pyspark.sql.functions import col, sum, when

# Create a DataFrame of null counts using select
null_df = flipkart_df.select([
    sum(when(col(c).isNull(), 1).otherwise(0)).alias(c)
    for c in flipkart_df.columns
])

null_df.show()




+---+-----+------+---------+--------+---------+---------+----------+-------+-------+-------+-------+-------+----------+
| id|title|Rating|maincateg|platform|actprice1|norating1|noreviews1|star_5f|star_4f|star_3f|star_2f|star_1f|fulfilled1|
+---+-----+------+---------+--------+---------+---------+----------+-------+-------+-------+-------+-------+----------+
|  0|    0|   203|       67|       0|        0|        0|         0|     68|      0|      0|      0|    186|         0|
+---+-----+------+---------+--------+---------+---------+----------+-------+-------+-------+-------+-------+----------+



In [0]:
# calculate and display the category name, number of fulfilled, and unfulfilled products
from pyspark.sql.functions import col,count,when
flipkart_df.groupBy("maincateg").agg(
    count(when(col("fulfilled1") == 0, True)).alias("Not Fulfilled Count"),
    count(when(col("fulfilled1") == 1, True)).alias("Fulfilled Count")
).show()


+---------+-------------------+---------------+
|maincateg|Not Fulfilled Count|Fulfilled Count|
+---------+-------------------+---------------+
|     null|                 27|             40|
|      Men|                793|           1352|
|    Women|               1254|           1778|
+---------+-------------------+---------------+



In [0]:
# Display Count of products per category
from pyspark.sql.functions import count
flipkart_df.groupBy("maincateg").agg(count("title")).show()



+---------+------------+
|maincateg|count(title)|
+---------+------------+
|     null|          67|
|      Men|        2145|
|    Women|        3032|
+---------+------------+



In [0]:
# Display Average rating per category
from pyspark.sql.functions import avg
flipkart_df.groupBy("maincateg").agg(avg("Rating")).show()


+---------+------------------+
|maincateg|       avg(Rating)|
+---------+------------------+
|     null|             4.225|
|      Men|3.9662970106075246|
|    Women|4.0416582064297915|
+---------+------------------+



In [0]:
# Display Category with highest average number of reviews

from pyspark.sql.functions import avg
flipkart_df.groupBy("maincateg").agg(avg("noreviews1")).orderBy("avg(noreviews1)",ascending=False).show(1)

+---------+-----------------+
|maincateg|  avg(noreviews1)|
+---------+-----------------+
|      Men|759.6554778554779|
+---------+-----------------+
only showing top 1 row



In [0]:
# Display Bar chart of product count per category

# Use Databricks UI to visualize this as a bar chart
display(flipkart_df.groupBy("maincateg").agg(count("title")))


maincateg,count(title)
null,67
Men,2145
Women,3032


Databricks visualization. Run in Databricks to view.

Databricks visualization. Run in Databricks to view.

In [0]:
# Bar chart of average rating per category
from pyspark.sql.functions import avg
display(flipkart_df.groupBy("maincateg").agg(avg("Rating")))



maincateg,avg(Rating)
null,4.225
Men,3.9662970106075246
Women,4.0416582064297915


Databricks visualization. Run in Databricks to view.

In [0]:
# Display Bar chart of total number of reviews per category
from pyspark.sql.functions import sum
display(flipkart_df.dropna(subset=["noreviews1", "maincateg"]).groupBy("maincateg").agg(sum("noreviews1")))


maincateg,sum(noreviews1)
Men,1629461
Women,535290


Databricks visualization. Run in Databricks to view.

In [0]:
# Display product name and 5 star rating for those products which have highest 5 star rating
flipkart_df.groupBy("title").agg(max("star_5f")).orderBy("max(star_5f)",ascending=False).show()

+--------------------+------------+
|               title|max(star_5f)|
+--------------------+------------+
|Jio-13 Running Sh...|      151193|
|Running Shoes For...|      151193|
|Jio-13 Running Sh...|      151159|
|VERTIGO BLK RUNNI...|       66369|
|Cosko Sports Shoe...|       64987|
|    Men Olive Sandal|       61722|
|Men Brown, Beige ...|       61722|
|Oxygen Running Sh...|       55231|
|Boxer-303 Sneaker...|       54272|
|Mesh,Cricket, Bad...|       45128|
|Mesh,Cricket, Bad...|       45128|
|Mesh,Cricket, Bad...|       45128|
|Mesh,Cricket, Bad...|       45127|
|SOCKS Running Sho...|       43374|
|Running Shoes For...|       42503|
|Combo Pack of 2 C...|       35264|
|  SFG-204 Flip Flops|       35161|
|Climber Boots For...|       33445|
|Climber Boots For...|       33443|
|Casuals For Men  ...|       32724|
+--------------------+------------+
only showing top 20 rows

